In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/train_data.csv
!wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/test_data.csv 

--2021-12-12 16:24:45--  https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/train_data.csv
Resolving gitlab.dei.unipd.it (gitlab.dei.unipd.it)... 147.162.2.85
Connecting to gitlab.dei.unipd.it (gitlab.dei.unipd.it)|147.162.2.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3776 (3,7K) [text/plain]
Saving to: 'regression_dataset/train_data.csv.10'

     0K ...                                                   100%  154K=0,02s

2021-12-12 16:24:46 (154 KB/s) - 'regression_dataset/train_data.csv.10' saved [3776/3776]

--2021-12-12 16:24:46--  https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/test_data.csv
Resolving gitlab.dei.unipd.it (gitlab.dei.unipd.it)... 147.162.2.85
Connecting to gitlab.dei.unipd.it (gitlab.dei.unipd.it)|147.162.2.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3788 (3,7K) [text/plain]
Saving to: 'regression_dataset/test_data.csv.10'

     0K

In [3]:
train_data = pd.read_csv('regression_dataset/train_data.csv')
test_data = pd.read_csv('regression_dataset/test_data.csv')

In [4]:
class CsvDataset(Dataset):

    def __init__(self, csv_file, transform=None):
        
        self.transform = transform
        # Read the file and split the lines in a list
        with open(csv_file, 'r') as f:
            lines = f.read().split('\n')
        lines.pop(0)
        lines.pop(-1)
        # Get x and y values from each line and append to self.data
        self.data = []
        for line in lines:
            sample = line.split(',')
            self.data.append((float(sample[0]), float(sample[1])))
            # Now self.data contains all our dataset.
        # Each element of the list self.data is a tuple: (input, output)
    def __len__(self):
        # The length of the dataset is simply the length of the self.data list
        return len(self.data)

    def __getitem__(self, idx):
        # Our sample is the element idx of the list self.data
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

In [5]:
class ToTensor():
        """Convert sample to Tensors."""
        def __call__(self, sample):
            x, y = sample
            return (torch.tensor([x]).float(), torch.tensor([y]).float())

In [6]:
composed_transform = transforms.Compose([ToTensor()])


In [7]:
from torch.utils.data.sampler import SubsetRandomSampler

#Validation test
indices = list(range(len(train_data)))
np.random.shuffle(indices)

# 20% of the train set will be validation set
split = int(np.floor(0.2 * len(train_data)))

train_set = SubsetRandomSampler(indices[:split])
valid_set = SubsetRandomSampler(indices[split:]) 


In [8]:
train_dataset = CsvDataset('regression_dataset/train_data.csv', transform=composed_transform)
#valid_dataset = CsvDataset('regression_dataset/valid_temp.csv', transform=composed_transform)
test_dataset = CsvDataset('regression_dataset/test_data.csv', transform=composed_transform)

In [9]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)
### Define validation dataloader
#val_dataloader = DataLoader(train_dataset, sampler=valid_set, batch_size=4)
test_dataloader = DataLoader(test_dataset,  batch_size=len(test_dataset), shuffle=False, num_workers=0)

In [10]:
'''
class Net(nn.Module):
    
    def __init__(self, Ni, Nh1, Nh2, No):
        
        super().__init__()
        
        print('Network initialized')
        
        self.linear = nn.Sequential(
            nn.Linear(Ni, Nh1),
            nn.Sigmoid(),
            nn.Linear(Nh1, Nh2),
            nn.Sigmoid(),
            nn.Linear(Nh2, No)       
        )
    
    def forward(self, x, additional_out=False):
        self.linear(x)
        return x
'''
class Net(nn.Module):
    
    def __init__(self, Ni, Nh1, Nh2, No):
        """
        Ni - Input size
        Nh1 - Neurons in the 1st hidden layer
        Nh2 - Neurons in the 2nd hidden layer
        No - Output size
        """
        super().__init__()
        
        print('Network initialized')
        self.fc1 = nn.Linear(in_features=Ni, out_features=Nh1)
        self.fc2 = nn.Linear(in_features=Nh1, out_features=Nh2)
        self.out = nn.Linear(in_features=Nh2, out_features=No)
        self.act = nn.Sigmoid()
        
    def forward(self, x, additional_out=False):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.out(x)
        return x

In [11]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

Training device: cpu


In [12]:
torch.manual_seed(0)
Ni = 1
Nh1 = 128
Nh2 = 256
No = 1
net = Net(Ni, Nh1, Nh2, No)
net.to(device)


Network initialized


Net(
  (fc1): Linear(in_features=1, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=1, bias=True)
  (act): Sigmoid()
)

In [13]:
loss_fn = nn.MSELoss()  
optimizer = optim.Adam(net.parameters(), lr = 1e-3)

In [14]:
### TRAINING LOOP
num_epochs = 300
train_loss_log = []
test_loss_log = []

for epoch_num in range(num_epochs):
    #print('#################')
    #print(f'# EPOCH {epoch_num}')
    #print('#################')

    ### TRAIN
    train_loss= []
    net.train() # Training mode (e.g. enable dropout, batchnorm updates,...)
    for sample_batched in train_dataloader:
        # Move data to device
        x_batch = sample_batched[0].to(device)
        label_batch = sample_batched[1].to(device)

        # Forward pass
        out = net(x_batch)

        # Compute loss
        loss = loss_fn(out, label_batch)

        # Backpropagation
        net.zero_grad()
        loss.backward()

        # Update the weights
        optimizer.step()

        # Save train loss for this batch
        loss_batch = loss.detach().cpu().numpy()
        train_loss.append(loss_batch)
        

    # Save average train loss
    train_loss = np.mean(train_loss)
    print(f"AVERAGE TRAIN LOSS: {train_loss}")
    train_loss_log.append(train_loss)
    
    ### VALIDATION
    test_loss= []
    net.eval() # Evaluation mode (e.g. disable dropout, batchnorm,...)
    with torch.no_grad(): # Disable gradient tracking
        for sample_batched in test_dataloader:
            # Move data to device
            x_batch = sample_batched[0].to(device)
            label_batch = sample_batched[1].to(device)

            # Forward pass
            out = net(x_batch)

            # Compute loss
            loss = loss_fn(out, label_batch)

            # Save val loss for this batch
            loss_batch = loss.detach().cpu().numpy()
            test_loss.append(loss_batch)

        # Save average validation loss
        test_loss = np.mean(test_loss)
        print(f"AVERAGE TEST LOSS: {np.mean(test_loss)}")
        test_loss_log.append(test_loss)

AVERAGE TRAIN LOSS: 4.4477996826171875
AVERAGE TEST LOSS: 3.0139875411987305
AVERAGE TRAIN LOSS: 2.819979190826416
AVERAGE TEST LOSS: 2.843344211578369
AVERAGE TRAIN LOSS: 2.6361396312713623
AVERAGE TEST LOSS: 2.94181489944458
AVERAGE TRAIN LOSS: 2.7298548221588135
AVERAGE TEST LOSS: 3.043100357055664
AVERAGE TRAIN LOSS: 2.663914203643799
AVERAGE TEST LOSS: 2.9221081733703613
AVERAGE TRAIN LOSS: 2.669382095336914
AVERAGE TEST LOSS: 2.862778902053833
AVERAGE TRAIN LOSS: 2.6570284366607666
AVERAGE TEST LOSS: 2.933281660079956
AVERAGE TRAIN LOSS: 2.650756597518921
AVERAGE TEST LOSS: 2.8703453540802
AVERAGE TRAIN LOSS: 2.661008834838867
AVERAGE TEST LOSS: 2.960939884185791
AVERAGE TRAIN LOSS: 2.619123935699463
AVERAGE TEST LOSS: 2.8579788208007812
AVERAGE TRAIN LOSS: 2.6482303142547607
AVERAGE TEST LOSS: 2.885441780090332
AVERAGE TRAIN LOSS: 2.676926612854004
AVERAGE TEST LOSS: 2.850956916809082
AVERAGE TRAIN LOSS: 2.5980658531188965
AVERAGE TEST LOSS: 2.8091113567352295
AVERAGE TRAIN LOSS

In [16]:
all_inputs = []
all_outputs = []
all_labels = []
net.eval() # Evaluation mode (e.g. disable dropout)
with torch.no_grad(): # Disable gradient tracking
    for sample_batched in test_dataloader:
        # Move data to device
        x_batch = sample_batched[0].to(device)
        label_batch = sample_batched[1].to(device)
        # Forward pass
        out = net(x_batch)
        # Save outputs and labels
        all_inputs.append(x_batch)
        all_outputs.append(out)
        all_labels.append(label_batch)
# Concatenate all the outputs and labels in a single tensor
all_inputs  = torch.cat(all_inputs)
all_outputs = torch.cat(all_outputs)
all_labels  = torch.cat(all_labels)

test_loss = loss_fn(all_outputs, all_labels)
print(f"AVERAGE TEST LOSS: {test_loss}")

AVERAGE TEST LOSS: 0.2634608745574951


In [92]:
X = np.zeros(len(train_data), dtype=np.float32)

y = np.zeros(len(train_data), dtype=np.float32)

#store the training /test data into ndarrays

for sample_index in range(len(train_data)):
    X[sample_index] = train_data.iloc[sample_index]['input']
    y[sample_index] = train_data.iloc[sample_index]['label']

X = X.reshape((len(train_data),1))
y = y.reshape((len(train_data),1))

X = torch.from_numpy(X)
y = torch.from_numpy(y)

print(X.shape)


torch.Size([100, 1])


In [86]:
X = X.reshape((len(train_data),1))
y = y.reshape((len(train_data),1))

X = torch.from_numpy(X)
y = torch.from_numpy(y)

print(X.shape)


torch.Size([100, 1])


In [137]:
from skorch import NeuralNetRegressor

net_skorch = NeuralNetRegressor(
    Net(Ni, Nh1, Nh2, No),
    max_epochs=300,
    lr=0.001,
    batch_size = 4,
    optimizer = optim.Adam
#            device='cuda',  # uncomment this to train with CUDA
)

Network initialized


In [132]:
print(X[0][0])

tensor(-4.7879)


In [133]:
net_skorch.fit(X, y)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.7405       10.5492  0.0811
      2        4.0107        5.9091  0.0922
      3        2.8802        4.3897  0.0780
      4        2.6328        4.0407  0.0908
      5        2.6260        3.7506  0.0799
      6        2.6040        3.6838  0.1002
      7        2.6069        3.6628  0.0914
      8        2.6003        3.6583  0.0893
      9        2.5910        3.6606  0.0788
     10        2.5803        3.6638  0.1041
     11        2.5688        3.6677  0.0804
     12        2.5569        3.6722  0.0902
     13        2.5446        3.6773  0.0834
     14        2.5318        3.6832  0.0980
     15        2.5185        3.6898  0.0800
     16        2.5047        3.6971  0.0804
     17        2.4903        3.7052  0.0978
     18        2.4753        3.7139  0.0778
     19        2.4596        3.7234  0.0797
     20        2.4433        3.7337  0.1067
     21        2.4262        3.7

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=1, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=1, bias=True)
    (act): Sigmoid()
  ),
)

In [134]:
y_pred = net_skorch.predict(X)

In [135]:
loss = net_skorch.get_loss(torch.from_numpy(y_pred), y)

In [136]:
loss

tensor(0.4178)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'lr': [0.01, 0.001, 0.0001],
    'max_epochs': list(range(50,300,50)),
    'batch_size' : list(range(2,10,2))
}
gs = GridSearchCV(net_skorch, params, refit=False, cv=3, scoring='neg_root_mean_squared_error')
# cv specifies the number of folds in a (Stratified)KFold

gs.fit(X.cpu(), y.cpu())
print(gs.best_score_, gs.best_params_)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.0731        1.4684  0.1499
      2        5.3133        8.1300  0.1200
      3        7.7844        1.2572  0.0998
      4        4.8227        2.4631  0.1095
      5        4.2289        4.5875  0.1071
      6        3.8306        3.8364  0.1100
      7        3.3822        2.2078  0.1264
      8        2.6675        1.6786  0.0963
      9        2.2432        1.6042  0.1128
     10        1.4651        1.6818  0.1183
     11        1.1741        1.7041  0.1170
     12        1.4150        1.6016  0.1165
     13        1.6745        1.4092  0.1270
     14        1.3068        1.3850  0.1201
     15        0.9642        1.4612  0.1028
     16        0.8576        1.5195  0.1128
     17        0.9557        1.5315  0.1129
     18        1.0913        1.4900  0.0989
     19        1.2809        1.3971  0.1206
     20        1.3717        1.2944  0.1107
     21        1.0875        1.3